Dependencies:
* matplotlib
* numpy
* scikit-learn
* pandas

In [1]:
%pylab inline
import pandas as pd
import json
import os
import glob

Populating the interactive namespace from numpy and matplotlib


# Dataset

In [2]:
# choose one of the two datasets
#dataset = "dataset/UC6_O3_lateral"
dataset = "dataset/UC4_O2_spherical_c_left_top_corner"
selected_features = [
    "finger_angle_index", "finger_angle_mrl", "finger_angle_thumb",
    "finger_tangential_mrl", "finger_normal_index", "finger_tangential_index",
    "finger_tangential_thumb", "finger_normal_thumb", "finger_normal_mrl"
]

In [3]:
def load_metadata(filename):
    with open(filename, "r") as f:
        return json.load(f)


def load_measurements(metadata):
    path = os.path.join(metadata["path"], metadata["sensor_filename"])
    return pd.read_csv(path, index_col=0)

For each grasp we stored some metadata. The most important for you is the label.

In [4]:
metadata = load_metadata(list(glob.glob(os.path.join(dataset, "*_failure_1.json")))[0])

In [5]:
metadata

{'label': 'failure',
 'sensor_filename': 'UC4_O2_failure_1.csv',
 'path': 'dataset/UC4_O2_spherical_c_left_top_corner',
 'p': [-0.0576, 0.1588, 0.4308],
 'rpy': [-0.8179, 0.3213, -3.123],
 'fraction': 0.9455,
 'grasp_typ': 'spherical',
 'object': 'UC4_O2',
 'orientation': 'c',
 'grasp_location': 'left top corner',
 'hand_calibration': 'MNS9'}

For the metadata we can also load the corresponding sensor measurements. Note that each sample has a different length!

In [6]:
measurements = load_measurements(metadata)[selected_features]

In [7]:
measurements

finger_angle_index  finger_angle_mrl  finger_angle_thumb  \
0              0.000000         -0.276250           -0.057961   
1              0.000000         -0.276250           -0.057961   
2              0.000000         -0.276250           -0.057961   
3              0.000000         -0.276250           -0.057961   
4              0.000000         -0.276250           -0.057961   
..                  ...               ...                 ...   
557            0.720113          0.937497            0.809848   
558            0.720113          0.937497            0.809848   
559            0.720113          0.937497            0.809848   
560            0.720113          0.937497            0.809848   
561            0.720113          0.937497            0.809848   

     finger_tangential_mrl  finger_normal_index  finger_tangential_index  \
0                     -1.0                 -1.0                     -1.0   
1                     -1.0                 -1.0                     -1.0   
2                     -1.0                 -1.0                     -1.0   
3                     -1.0                 -1.0                     -1.0   
4                     -1.0                 -1.0                     -1.0   
..                     ...                  ...                      ...   
557                   -5.0                 -7.0                     -4.0   
558                   -5.0                 -7.0                     -4.0   
559                   -5.0                 -7.0                     -4.0   
560                   -5.0                 -7.0                     -4.0   
561                   -5.0                 -7.0                     -4.0   

     finger_tangential_thumb  finger_normal_thumb  finger_normal_mrl  
0                        0.0                  2.0                0.0  
1                        0.0                  2.0                0.0  
2                        0.0                  2.0                0.0  
3                        0.0                  2.0                0.0  
4                        0.0                  2.0                0.0  
..                       ...                  ...                ...  
557                      8.0                -67.0               -9.0  
558                      8.0                -67.0               -9.0  
559                      8.0                -67.0               -9.0  
560                      8.0                -67.0               -9.0  
561                      8.0                -67.0               -9.0  

[562 rows x 9 columns]

In [8]:
def load_dataset(dataset, selected_features):
    metadata_filenames = list(sorted(glob.glob(os.path.join(dataset, "*.json"))))
    y = []
    X = []
    for fn in metadata_filenames:
        metadata = load_metadata(fn)
        measurements = load_measurements(metadata)[selected_features]
        X.append(measurements.to_numpy())
        y.append(metadata["label"] == "success_and_stable")
    return X, np.array(y)

In [9]:
X_raw, y_raw = load_dataset(dataset, selected_features)

# Preprocessing

Before applying machine learning, make sure to convert each sample to a vector of the same length!

In [10]:
def preprocess(X, y, n_steps):
    """Proprocess dataset so that all input vectors have the same length.

    Parameters
    ----------
    X : list
        Samples, each entry is an array with shape (n_measurements, n_features),
        where n_measurements is different for each sample.

    y : array, shape (n_samples,)
        Labels

    n_steps : int
        Number of steps per sample that should be kept.

    Returns
    -------
    X : array, shape (n_samples, n_steps * n_features)
        Samples with equal lengths as 2D array.

    n_steps : int
        Number of steps per sample that should be kept.
    """
    return X, y

In [13]:
n_history = 10

In [14]:
X_train, y_train = preprocess(X_raw, y_raw, n_history)

# Machine Learning

Now it is time to do machine learning. :)

**Try to obtain the best classifier possible.**

You can use any ML library, but we recommend to use [sklearn](https://scikit-learn.org/stable/modules/classes.html). In the end, we will test the classifier's performance on a test set that you don't have yet. So make sure to not overfit the dataset that you have! How would you do that?

# Evaluation on Test Set

In [ ]:
X_test_raw, y_test_raw = load_dataset(dataset + "_test", selected_features)
X_test, y_test = preprocess(X_test_raw, y_test_raw, n_history)
grid.score(X_test, y_test)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

cm = confusion_matrix(grid.predict(X_test), y_test)
ConfusionMatrixDisplay(cm, display_labels=["Stable", "Unstable"]).plot()